# **Computing $w(\theta)$ for a given cosmology and a set of redshift distributions**

This notebook provides a step-by-step guide to calculate the theoretical angular correlation function $w(\theta)$ based on a given cosmology and redshift distributions.

---

1. **Full integrals**:  
   The code computes $w(\theta)$ without using simplifications like the Limber or flat-sky approximations.

2. **Parallel processing**:  
   By default, this notebook uses **256 CPUs** to speed up calculations.

3. **Optional no-wiggle template**:  
   You can generate a template without BAO wiggles to evaluate the significance of the BAO detection when running the BAO fits.

---


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
code_path = f"{os.environ['PSCRATCH']}/BAOfit_wtheta"
save_path = f"{os.environ['PSCRATCH']}/BAOfit_wtheta"
sys.path.append(code_path)
from utils_template import TemplateInitializer, PowerSpectrumMultipoles, CorrelationFunctionMultipoles, WThetaCalculator, CellCalculator
import itertools

# # Option A. DESY6-related stuff
# dataset_list = [
#     "DESY6",
#     "DESY6_COLA"
# ]
# include_wiggles_list = ["y", "n"]
# cosmology_template_list = ["planck_old", "mice_old"]

# Option B. DESI-related stuff
dataset_list = [
    # "DESIY1_LRG_EZ_ffa_deltaz0.028", # since all of them have the same n(z) (just a top-hat function), run it for one and copy/paste for the rest
    "DESIY1_LRG_EZ_ffa_deltaz0.02", # since all of them have the same n(z) (just a top-hat function), run it for one and copy/paste for the rest
]
include_wiggles_list = ["y", "n"]
cosmology_template_list = ["desifid"]

for dataset, include_wiggles, cosmology_template in itertools.product(dataset_list, include_wiggles_list, cosmology_template_list):

    if dataset == "DESY6":
        nz_flag = "fid"
    elif dataset == "DESY6_COLA":
        nz_flag = "mocks"
    elif "DESIY1" in dataset:
        nz_flag = "mocks"

    # 1. Arguments
    class Args:
        def __init__(self):
            self.include_wiggles = include_wiggles
            self.dataset = dataset
            self.nz_flag = nz_flag
            self.cosmology_template = cosmology_template
            self.code_path = code_path
            self.save_path = save_path
    args = Args()
    args.include_wiggles = "" if args.include_wiggles == "y" else "_nowiggles"
    
    # 2. Numerical resolution of the calculation. The larger these numbers are, the more accurate the template will be
    if "DESIY1" in args.dataset:
        Nk, Nmu, Nr, Nz, Ntheta = 2*10**5, 5*10**4, 5*10**4, 10**2, 10**3 # I reduced Nz since the n(z) is just a top-hat function in a thin redshift bin
    else:
        Nk, Nmu, Nr, Nz, Ntheta = 2*10**5, 5*10**4, 5*10**4, 10**3, 10**3 # these are the settings I used for DES Y6 BAO
    
    # 3. Initialize the template
    template_initializer = TemplateInitializer(
        include_wiggles=args.include_wiggles,
        dataset=args.dataset,
        nz_flag=args.nz_flag,
        cosmology_template=args.cosmology_template,
        Nk=Nk,
        Nmu=Nmu,
        Nr=Nr,
        Nz=Nz,
        Ntheta=Ntheta,
        verbose=True,
        use_multiprocessing=True,
        n_cpu=256,
        code_path=args.code_path,
        save_path=args.save_path,
    )
    
    # 4. Calculation of pk_ell
    pk_calculator = PowerSpectrumMultipoles(
        template_initializer=template_initializer,
    )
    for bin_z in range(template_initializer.nbins):
        pk_calculator.compute_pk_ell(bin_z)
    
    # 5. Calculation of xi_ell
    xi_calculator = CorrelationFunctionMultipoles(
        template_initializer=template_initializer,
    )
    for bin_z in range(template_initializer.nbins):
        xi_calculator.compute_xi_ell(bin_z)
    
    # 6. Calculation of w(theta)
    wtheta_calculator = WThetaCalculator(
        template_initializer=template_initializer,
    )
    for bin_z in range(template_initializer.nbins):
        wtheta_calculator.compute_wtheta(bin_z)
    
    # # 7. Calculation of C_ell (optional!)
    # C_ell_calculator = CellCalculator(
    #     template_initializer=template_initializer,
    # )
    # for bin_z in range(template_initializer.nbins):
    #     C_ell_calculator.compute_C_ell(bin_z)
